# Criar a estrutura do arquivo pokedex.proto

In [1]:
proto_file = """
syntax = "proto3";

message Pokemon {
    message Type {
        string name = 1;
        string url = 2;
    }

    message Types {
        uint32 slot = 1;
        Type type = 2;
    }

    uint32 id = 1;
    string name = 2;
    uint32 order = 3;
    uint32 height = 4;
    uint32 weight = 5;
    uint32 base_experience = 6;
    repeated Types types = 7;
}

message Pokedex {
    repeated Pokemon pokemon = 1;
}
"""

with open('pokedex.proto', '+w') as f:
    f.write(proto_file)

# Build do arquivo pokedex.proto

In [2]:
%%sh
INPUT_DIR="/content"
INPUT_FILE="$INPUT_DIR/pokedex.proto"
OUTPUT_DIR="/content"

protoc --proto_path=$INPUT_DIR --python_out=$OUTPUT_DIR $INPUT_FILE

# Importação de pacotes

In [3]:
import sys
sys.path.append("/content")

import os
import json 
from datetime import datetime
from io import StringIO, BytesIO

import pandas as pd
import requests
from google.protobuf.json_format import MessageToJson
from pokedex_pb2 import Pokedex

# Criar lista de pokemons realizando consumo da pokeapi

In [4]:
def clean_data(pokemon_json):
    for key in ['abilities', 'forms', 'game_indices', 'held_items', 'moves', 
                'sprites', 'stats', 'past_types', 'location_area_encounters', 
                'is_default', 'species']:
        del pokemon_json[key]
    return pokemon_json

def get_pokemons(first_id=1, last_id=1):
    pokemons = []
    for id in range(first_id, last_id + 1):
        response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{id}')
        pokemons.append(clean_data(response.json()))
    return pokemons

pokemons = get_pokemons(1, 151)
print(f"{StringIO(json.dumps(pokemons)).seek(0, os.SEEK_END)} bytes")


34859 bytes


# Serializar lista de pokemons para objeto `pokedex` e escrever arquivo `pokedex.data`

In [5]:
pokedex = Pokedex(pokemon=pokemons)
print(pokedex.ByteSize(), 'bytes')

with open('pokedex.data', "wb") as f:
    f.write(pokedex.SerializeToString())

14175 bytes


# Ler arquivo `pokedex.data`

In [6]:
pokedex = Pokedex()
with open('pokedex.data', "rb") as f:
    pokedex.ParseFromString(f.read())
print(pokedex.ByteSize(), 'bytes')
print('\nSample Pokemon\n==============')
print(pokedex.pokemon[1])


14175 bytes

Sample Pokemon
id: 2
name: "ivysaur"
order: 2
height: 10
weight: 130
base_experience: 142
types {
  slot: 1
  type {
    name: "grass"
    url: "https://pokeapi.co/api/v2/type/12/"
  }
}
types {
  slot: 2
  type {
    name: "poison"
    url: "https://pokeapi.co/api/v2/type/4/"
  }
}



# Converter `proto` para dataframe `pandas`

In [7]:
# Método 1
df = pd.DataFrame.from_dict(
        json.loads(MessageToJson(pokedex)
    ), orient='columns'
)
display(df.head(5))

,pokemon
0,"{'id': 1, 'name': 'bulbasaur', 'order': 1, 'he..."
1,"{'id': 2, 'name': 'ivysaur', 'order': 2, 'heig..."
2,"{'id': 3, 'name': 'venusaur', 'order': 3, 'hei..."
3,"{'id': 4, 'name': 'charmander', 'order': 5, 'h..."
4,"{'id': 5, 'name': 'charmeleon', 'order': 6, 'h..."


In [8]:
# Método 2
df = pd.read_json(
    StringIO(MessageToJson(pokedex))
)
display(df.head(5))

,pokemon
0,"{'id': 1, 'name': 'bulbasaur', 'order': 1, 'he..."
1,"{'id': 2, 'name': 'ivysaur', 'order': 2, 'heig..."
2,"{'id': 3, 'name': 'venusaur', 'order': 3, 'hei..."
3,"{'id': 4, 'name': 'charmander', 'order': 5, 'h..."
4,"{'id': 5, 'name': 'charmeleon', 'order': 6, 'h..."
